In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cab

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import warnings
import json 
import pickle
warnings.filterwarnings('ignore')
from sklearn.preprocessing import KBinsDiscretizer

In [3]:
import time
import numpy as np 
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import re
# import math
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# from lightgbm import LGBMRegressor, LGBMClassifier
# from xgboost import XGBRegressor, XGBClassifier
# from catboost import CatBoostRegressor, CatBoostClassifier
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cab

# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
# from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
# from sklearn.linear_model import LogisticRegression, SGDClassifier
# from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
# from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
# from sklearn import metrics
# from sklearn.svm import SVC
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
# from collections import defaultdict, Counter
import warnings
import json 
import pickle
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pickle
import random
from tqdm import tqdm
# import sentence_transformers 
# from sklearn.preprocessing import KBinsDiscretizer
# from sentence_transformers import SentenceTransformer

# from gensim.models import Word2Vec

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import random_split, DataLoader
from torch.utils.data import Dataset
import argparse
from utils import set_random_seed, get_logger, ensure_dir, str2bool, str2float
from data import NNDataset, NNDatasetV2, NNDatasetV3
from model import MatchModel, BaseModel, MatchModelV2, BinaryModel
from sklearn import metrics

seed = 2023
Fold = 0
set_random_seed(seed)
task = 'task1'
feature = 'v3'
dense_norm = True
recall = 'window'
len_candidate_set = 10
add_title = False
add_desc = False
add_w2v = False
w2v_vector_size = 128

model_name = 'LGBFold{}'.format(Fold)
loc2id = {'DE': 0, 'JP': 1, 'UK': 2, 'ES': 3, 'FR': 4, 'IT': 5}

config = locals()

# 加载必要的数据

exp_id = None
if exp_id is None:
    exp_id = int(random.SystemRandom().random() * 100000)
    config['exp_id'] = exp_id

logger = get_logger(config)
logger.info('Exp_id {}'.format(exp_id))
logger.info(config)

logger.info('read data')

titles_embedding = np.load('./data/titles_embedding.npy')
descs_embedding = np.load('./data/descs_embedding.npy')
logger.info('titles_embedding: {}'.format(titles_embedding.shape))
logger.info('descs_embedding: {}'.format(descs_embedding.shape))

product2id = json.load(open('data/product2id.json', 'r'))
id2product = json.load(open('data/id2product.json', 'r'))
id2product = {int(k): v for k, v in id2product.items()}
logger.info('product2id: {}'.format(len(product2id)))
logger.info('id2product: {}'.format(len(id2product)))

word2vec_embedding = np.load('./data/word2vec_embedding.npy')
logger.info('word2vec_embedding: {}'.format(word2vec_embedding.shape))

df_train_encoded = pd.read_csv('data/df_train_encoded.csv')
if task == 'task1':
    df_test_encoded = pd.read_csv('data/df_test_encoded_phase2.csv')
elif task == 'task2':
    df_test_encoded = pd.read_csv('data/df_test_encoded_phase2_onlytask2.csv')

if feature.lower() == 'v1':
    products_encoded = pd.read_csv('./data/products_encoded.csv')
    num_features = ['price', 'len_title', 'len_desc']
elif feature.lower() == 'v2':
    products_encoded = pd.read_csv('./data/products_encoded_newfeature.csv')
    num_features = ['price']
elif feature.lower() == 'v3':
    products_encoded = pd.read_csv('./data/products_encoded_phase2_V3.csv')
    num_features = ['price']

logger.info('df_train_encoded: {}'.format(df_train_encoded.shape))
logger.info('df_test_encoded: {}'.format(df_test_encoded.shape))
logger.info('products_encoded: {}'.format(products_encoded.shape))

if dense_norm:
    logger.info('MinMaxScaler Norm products_num_feas')
    mms = MinMaxScaler(feature_range=(0,1))
    products_encoded[num_features] = mms.fit_transform(products_encoded[num_features])
for fe in num_features:
    products_encoded[fe] = products_encoded[fe].astype('float32')
    assert products_encoded[fe].dtypes == 'float32'

id_count = products_encoded.shape[0]

if task == 'task1':
    train_preds_encoded = pickle.load(open('./data/train_preds_{}_one_phase2_noleak_encoded.pkl'.format(recall), 'rb'))  # (len_train, 100)
    test_preds_encoded = pickle.load(open('./data/test_preds_{}_one_phase2_encoded.pkl'.format(recall), 'rb'))  # (len_test, 100)
    logger.info('./data/train_preds_{}_one_phase2_noleak_encoded.pkl'.format(recall))
    logger.info('train_preds_encoded: {}'.format(len(train_preds_encoded)))
    logger.info('./data/test_preds_{}_one_phase2_encoded.pkl'.format(recall))
    logger.info('test_preds_encoded: {}'.format(len(test_preds_encoded)))
elif task == 'task2':
    train_preds_encoded = pickle.load(open('./data/train_preds_{}_one_onlytask2_phase2_noleak_encoded.pkl'.format(recall), 'rb'))  # (len_train, 100)
    test_preds_encoded = pickle.load(open('./data/test_preds_{}_one_onlytask2_phase2_encoded.pkl'.format(recall), 'rb'))  # (len_test, 100)
    logger.info('./data/train_preds_{}_one_onlytask2_phase2_noleak_encoded.pkl'.format(recall))
    logger.info('train_preds_encoded: {}'.format(len(train_preds_encoded)))
    logger.info('./data/test_preds_{}_one_onlytask2_phase2_encoded.pkl'.format(recall))
    logger.info('test_preds_encoded: {}'.format(len(test_preds_encoded)))

logger.info('Cutting the candidate_set to {}'.format(len_candidate_set))
# TODO: 可以改成保障一个正样本，补充9个负样本
cut_train_preds_encoded = [lst[:len_candidate_set] for lst in tqdm(train_preds_encoded, total=len(train_preds_encoded))]
df_train_encoded['recall'] = cut_train_preds_encoded
cut_test_preds_encoded = [lst[:len_candidate_set] for lst in tqdm(test_preds_encoded, total=len(test_preds_encoded))]
df_test_encoded['recall'] = cut_test_preds_encoded

logger.info('Eval the prev_items')
df_train_encoded['prev_items'] = df_train_encoded['prev_items'].apply(eval)
df_test_encoded['prev_items'] = df_test_encoded['prev_items'].apply(eval)
df_train_encoded['last_item_2'] = df_train_encoded['prev_items'].apply(lambda x: x[-2])
df_test_encoded['last_item_2'] = df_test_encoded['prev_items'].apply(lambda x: x[-2])

logger.info('Load Hand-made Seq Features')

if feature.lower() == 'v1':
    df_train_seqs_feas_all = pd.read_csv('data/df_train_seqs_feas_all.csv')  # 29维特征
    if task == 'task1':
        df_test_seqs_feas_all = pd.read_csv('data/df_test_seqs_feas_all_phase2.csv')
    elif task == 'task2':
        df_test_seqs_feas_all = pd.read_csv('data/df_test_seqs_feas_all_phase2_onlytask2.csv')
    dense_bins = 10
elif feature.lower() == 'v2':
    df_train_seqs_feas_all = pd.read_csv('data/df_train_seqs_feas_all_newfeature.csv')  # 24维特征
    if task == 'task1':
        df_test_seqs_feas_all = pd.read_csv('data/df_test_seqs_feas_all_newfeature_phase2.csv')
    elif task == 'task2':
        df_test_seqs_feas_all = pd.read_csv('data/df_test_seqs_feas_all_newfeature_phase2_onlytask2.csv')
    dense_bins = 100
elif feature.lower() == 'v3':
    df_train_seqs_feas_all = pd.read_csv('data/df_train_seqs_feas_all_phase2_V3.csv')  # 36维特征
    if task == 'task1':
        df_test_seqs_feas_all = pd.read_csv('data/df_test_seqs_feas_all_phase2_V3.csv')
    elif task == 'task2':
        df_test_seqs_feas_all = pd.read_csv('data/df_test_seqs_feas_all_phase2_V3_onlytask2.csv')
    dense_bins = 100

logger.info('df_train_seqs_feas_all: {}'.format(df_train_seqs_feas_all.shape))
logger.info('df_test_seqs_feas_all: {}'.format(df_test_seqs_feas_all.shape))
seqs_cat_feas = [f for f in df_train_seqs_feas_all.columns if 'NUNIQUE' in f or 'COUNT' in f or 'encode_' in f]
seqs_num_feas = [f for f in df_train_seqs_feas_all.columns if f not in seqs_cat_feas]
logger.info('seqs_cat_feas: {}'.format(seqs_cat_feas))
logger.info('seqs_num_feas: {}'.format(seqs_num_feas))

if dense_norm:
    logger.info('MinMaxScaler Norm seqs_num_feas')
    mms = MinMaxScaler(feature_range=(0,1))
    df_train_seqs_feas_all[seqs_num_feas] = mms.fit_transform(df_train_seqs_feas_all[seqs_num_feas])
    df_test_seqs_feas_all[seqs_num_feas] = mms.fit_transform(df_test_seqs_feas_all[seqs_num_feas])

for fe in seqs_num_feas:
    df_train_seqs_feas_all[fe] = df_train_seqs_feas_all[fe].astype('float32')
    df_test_seqs_feas_all[fe] = df_test_seqs_feas_all[fe].astype('float32')

df_train_all = pd.concat([df_train_encoded, df_train_seqs_feas_all], axis=1)
df_test_all = pd.concat([df_test_encoded, df_test_seqs_feas_all], axis=1)
logger.info('df_train_all: {}'.format(df_train_all.shape))
logger.info('df_test_all: {}'.format(df_test_all.shape))

df_train_all_exploded = df_train_all.explode('recall')
df_test_all_exploded = df_test_all.explode('recall')

df_train_all_exploded['label'] = df_train_all_exploded['next_item'] == df_train_all_exploded['recall']
df_test_all_exploded['label'] = df_test_all_exploded['next_item'] == df_test_all_exploded['recall']

df_train_all_exploded['index'] = df_train_all_exploded.index 
df_test_all_exploded['index'] = df_test_all_exploded.index 

logger.info('df_train_all_exploded: {}'.format(df_train_all_exploded.shape))
logger.info('df_test_all_exploded: {}'.format(df_test_all_exploded.shape))

logger.info('df_train_all_exploded.label.sum: {}'.format(df_train_all_exploded['label'].sum()))
logger.info('df_test_all_exploded.label.sum: {}'.format(df_test_all_exploded['label'].sum()))

df_train_encoded_exploded = df_train_all_exploded[['prev_items', 'next_item', 'locale', 'recall', 'label', 'index']]
df_train_seqs_cat_feas = df_train_all_exploded[seqs_cat_feas]
df_train_seqs_num_feas = df_train_all_exploded[seqs_num_feas]
df_test_encoded_exploded = df_test_all_exploded[['prev_items', 'next_item', 'locale', 'recall', 'label', 'index']]
df_test_seqs_cat_feas = df_test_all_exploded[seqs_cat_feas]
df_test_seqs_num_feas = df_test_all_exploded[seqs_num_feas]

logger.info('df_train_encoded_exploded: {}'.format(df_train_encoded_exploded.shape))
logger.info('df_train_seqs_cat_feas: {}'.format(df_train_seqs_cat_feas.shape))
logger.info('df_train_seqs_num_feas: {}'.format(df_train_seqs_num_feas.shape))
logger.info('df_test_encoded_exploded: {}'.format(df_test_encoded_exploded.shape))
logger.info('df_test_seqs_cat_feas: {}'.format(df_test_seqs_cat_feas.shape))
logger.info('df_test_seqs_num_feas: {}'.format(df_test_seqs_num_feas.shape))

if task == 'task1':
    df_test = pd.read_csv('data/phase2/sessions_test_task1.csv')
    logger.info('df_test Task 1 Phase 2: {}'.format(df_test.shape))
elif task == 'task2':
    df_test = pd.read_csv('data/phase2/sessions_test_task2.csv')
    logger.info('df_test Task 2 Phase 2: {}'.format(df_test.shape))


2023-06-03 13:45:28,705 - INFO - Log directory: ./log
2023-06-03 13:45:28,706 - INFO - Exp_id 17581
2023-06-03 13:45:28,707 - INFO - {'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'df_train_all_exploded_add_last_recall', "import numpy as np \nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport warnings\nimport os\nimport re\nimport math\nimport plotly.express as px\nimport plotly.graph_objects as go\nfrom plotly.subplots import make_subplots\n\nfrom lightgbm import LGBMRegressor, LGBMClassifier\nfrom xgboost import XGBRegressor, XGBClassifier\nfrom catboost import CatBoostRegressor, CatBoostClassifier\nimport lightgbm as lgb\nimport xgboost as xgb\nimport catboost as cab\n\nfrom sklearn.preprocessing import LabelEncoder\nfrom sklearn.m

100%|██████████| 316972/316972 [00:10<00:00, 30032.84it/s] 

2023-06-03 13:47:12,163 - INFO - Eval the prev_items


2023-06-03 13:48:04,493 - INFO - Load Hand-made Seq Features
2023-06-03 13:48:13,551 - INFO - df_train_seqs_feas_all: (3606249, 36)
2023-06-03 13:48:13,552 - INFO - df_test_seqs_feas_all: (316972, 36)
2023-06-03 13:48:13,553 - INFO - seqs_cat_feas: ['idNUNIQUE', 'idCOUNT', 'encode_brandNUNIQUE', 'encode_brandCOUNT', 'encode_brandMYMODE', 'encode_colorNUNIQUE', 'encode_colorCOUNT', 'encode_colorMYMODE', 'encode_sizeNUNIQUE', 'encode_sizeCOUNT', 'encode_sizeMYMODE', 'encode_modelNUNIQUE', 'encode_modelCOUNT', 'encode_modelMYMODE', 'encode_materialNUNIQUE', 'encode_materialCOUNT', 'encode_materialMYMODE', 'encode_authorNUNIQUE', 'encode_authorCOUNT', 'encode_authorMYMODE', 'encode_priceMEAN', 'encode_priceSTD', 'encode_priceMIN', 'encode_priceMAX', 'encode_priceSUM', 'encode_pricePTP', 'encode_priceQUANTILE75', 'encode_priceQUANTILE25']
2023-06-03 13:48:13,553 - INFO - seqs_num_feas: ['priceMEAN', 'priceSTD', 'priceMIN', 'priceMAX', 'priceSUM', 'pricePTP', 'priceQUANTILE75', 'priceQUANTIL

In [4]:
# 序列特征如果是平均值，也很奇怪，因为都是类别特征，现在相当于只考虑last-item的特征，以及召回的recall的特征，预测label
# w2v desc title怎么加进去？维度太大了

In [5]:
df_train_encoded.head()

,prev_items,next_item,locale,last_item,recall,last_item_2
0,"[265193, 83226]",387776,0,83226,"[265193, 83226, 218958, 174133, 54056, 236419,...",265193
1,"[38788, 85634, 4132, 71046]",335301,0,71046,"[335301, 4132, 484264, 307774, 273700, 239289,...",4132
2,"[191882, 516876, 191882, 516876, 191882, 19188...",90141,0,516876,"[191882, 516876, 198906, 360214, 123364, 27777...",191882
3,"[136959, 261145, 31496, 136959, 261145, 31496,...",214540,0,469511,"[136959, 31496, 375995, 261145, 219917, 474070...",31496
4,"[291068, 410614, 4219]",338089,0,4219,"[355730, 410614, 435967, 210849, 43550, 346031...",410614


In [6]:
df_train_all_exploded['recall'] = df_train_all_exploded['recall'].astype('int64')
df_test_all_exploded['recall'] = df_test_all_exploded['recall'].astype('int64')

In [7]:
df_train_all_exploded_add_last_recall = pd.merge(df_train_all_exploded, products_encoded.add_suffix('_last_item'), left_on='last_item', right_on='id_last_item')
df_test_all_exploded_add_last_recall = pd.merge(df_test_all_exploded, products_encoded.add_suffix('_last_item'), left_on='last_item', right_on='id_last_item')

In [8]:
df_train_all_exploded_add_last_recall.drop('id_last_item', axis=1, inplace=True)
df_train_all_exploded_add_last_recall.drop('locale_last_item', axis=1, inplace=True)
df_test_all_exploded_add_last_recall.drop('id_last_item', axis=1, inplace=True)
df_test_all_exploded_add_last_recall.drop('locale_last_item', axis=1, inplace=True)

In [9]:
# 43 + 10 - 2 = 51

In [10]:
df_train_all_exploded_add_last_recall.shape, df_test_all_exploded_add_last_recall.shape

((36062490, 52), (3169720, 52))

In [11]:
df_train_all_exploded_add_last_recall = pd.merge(df_train_all_exploded_add_last_recall, products_encoded.add_suffix('_last_item_2'), left_on='last_item_2', right_on='id_last_item_2')
df_test_all_exploded_add_last_recall = pd.merge(df_test_all_exploded_add_last_recall, products_encoded.add_suffix('_last_item_2'), left_on='last_item_2', right_on='id_last_item_2')

In [12]:
df_train_all_exploded_add_last_recall.drop('id_last_item_2', axis=1, inplace=True)
df_train_all_exploded_add_last_recall.drop('locale_last_item_2', axis=1, inplace=True)
df_test_all_exploded_add_last_recall.drop('id_last_item_2', axis=1, inplace=True)
df_test_all_exploded_add_last_recall.drop('locale_last_item_2', axis=1, inplace=True)

In [13]:
df_train_all_exploded_add_last_recall.shape, df_test_all_exploded_add_last_recall.shape

((36062490, 60), (3169720, 60))

In [14]:
df_train_all_exploded_add_last_recall = pd.merge(df_train_all_exploded_add_last_recall, products_encoded.add_suffix('_recall'), left_on='recall', right_on='id_recall')
df_test_all_exploded_add_last_recall = pd.merge(df_test_all_exploded_add_last_recall, products_encoded.add_suffix('_recall'), left_on='recall', right_on='id_recall')

In [15]:
df_train_all_exploded_add_last_recall.drop('id_recall', axis=1, inplace=True)
df_train_all_exploded_add_last_recall.drop('locale_recall', axis=1, inplace=True)
df_test_all_exploded_add_last_recall.drop('id_recall', axis=1, inplace=True)
df_test_all_exploded_add_last_recall.drop('locale_recall', axis=1, inplace=True)

In [16]:
# 51 + 10 - 2 = 59

In [17]:
df_train_all_exploded_add_last_recall.shape, df_test_all_exploded_add_last_recall.shape

((36062490, 68), (3169720, 68))

In [18]:
len(df_train_all_exploded_add_last_recall.columns)

68

In [19]:
df_train_all_exploded_add_last_recall.columns

Index(['prev_items', 'next_item', 'locale', 'last_item', 'recall',
       'last_item_2', 'idNUNIQUE', 'idCOUNT', 'encode_brandNUNIQUE',
       'encode_brandCOUNT', 'encode_brandMYMODE', 'encode_colorNUNIQUE',
       'encode_colorCOUNT', 'encode_colorMYMODE', 'encode_sizeNUNIQUE',
       'encode_sizeCOUNT', 'encode_sizeMYMODE', 'encode_modelNUNIQUE',
       'encode_modelCOUNT', 'encode_modelMYMODE', 'encode_materialNUNIQUE',
       'encode_materialCOUNT', 'encode_materialMYMODE', 'encode_authorNUNIQUE',
       'encode_authorCOUNT', 'encode_authorMYMODE', 'priceMEAN', 'priceSTD',
       'priceMIN', 'priceMAX', 'priceSUM', 'pricePTP', 'priceQUANTILE75',
       'priceQUANTILE25', 'encode_priceMEAN', 'encode_priceSTD',
       'encode_priceMIN', 'encode_priceMAX', 'encode_priceSUM',
       'encode_pricePTP', 'encode_priceQUANTILE75', 'encode_priceQUANTILE25',
       'label', 'index', 'price_last_item', 'encode_brand_last_item',
       'encode_color_last_item', 'encode_size_last_item',
      

In [20]:
df_train_all_exploded_add_last_recall.dtypes

prev_items                object
next_item                  int64
locale                     int64
last_item                  int64
recall                     int64
                           ...  
encode_size_recall         int64
encode_model_recall        int64
encode_material_recall     int64
encode_author_recall       int64
encode_price_recall        int64
Length: 68, dtype: object

In [21]:
# cross fea
for f1 in tqdm(['encode_brandNUNIQUE',
                'encode_brandCOUNT',
                'encode_colorNUNIQUE',
                'encode_colorCOUNT',
                'encode_sizeNUNIQUE',
                'encode_sizeCOUNT',
                'encode_modelNUNIQUE',
                'encode_modelCOUNT',
                'encode_materialNUNIQUE',
                'encode_materialCOUNT',
                'encode_authorNUNIQUE',
                'encode_authorCOUNT'], total=12):
    for f2 in ['idNUNIQUE', 'idCOUNT']:
        df_train_all_exploded_add_last_recall[f'{f1}/{f2}'] = df_train_all_exploded_add_last_recall[f1] / df_train_all_exploded_add_last_recall[f2]
        df_test_all_exploded_add_last_recall[f'{f1}/{f2}'] = df_test_all_exploded_add_last_recall[f1] / df_test_all_exploded_add_last_recall[f2]

100%|██████████| 12/12 [00:07<00:00,  1.54it/s]


In [22]:
# cross fea
for f1 in tqdm(['price_recall'], total=1):
    for f2 in ['price_last_item', 'price_last_item_2']:
        df_train_all_exploded_add_last_recall[f'{f1}-{f2}'] = (df_train_all_exploded_add_last_recall[f1] - df_train_all_exploded_add_last_recall[f2])
        df_test_all_exploded_add_last_recall[f'{f1}-{f2}'] = (df_test_all_exploded_add_last_recall[f1] - df_test_all_exploded_add_last_recall[f2])

100%|██████████| 1/1 [00:00<00:00,  2.50it/s]


In [23]:
# cross fea
for f in ['price', 'brand', 'color', 'size', 'model', 'material', 'author']:
    for f1 in tqdm(['encode_{}_recall'.format(f)], total=1):
        for f2 in ['encode_{}_last_item'.format(f), 'encode_{}_last_item_2'.format(f)]:
            df_train_all_exploded_add_last_recall[f'{f1}=={f2}'] = (df_train_all_exploded_add_last_recall[f1] == df_train_all_exploded_add_last_recall[f2]).astype('int')
            df_test_all_exploded_add_last_recall[f'{f1}=={f2}'] = (df_test_all_exploded_add_last_recall[f1] == df_test_all_exploded_add_last_recall[f2]).astype('int')

100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


In [24]:
list(df_train_all_exploded_add_last_recall.columns)

['prev_items',
 'next_item',
 'locale',
 'last_item',
 'recall',
 'last_item_2',
 'idNUNIQUE',
 'idCOUNT',
 'encode_brandNUNIQUE',
 'encode_brandCOUNT',
 'encode_brandMYMODE',
 'encode_colorNUNIQUE',
 'encode_colorCOUNT',
 'encode_colorMYMODE',
 'encode_sizeNUNIQUE',
 'encode_sizeCOUNT',
 'encode_sizeMYMODE',
 'encode_modelNUNIQUE',
 'encode_modelCOUNT',
 'encode_modelMYMODE',
 'encode_materialNUNIQUE',
 'encode_materialCOUNT',
 'encode_materialMYMODE',
 'encode_authorNUNIQUE',
 'encode_authorCOUNT',
 'encode_authorMYMODE',
 'priceMEAN',
 'priceSTD',
 'priceMIN',
 'priceMAX',
 'priceSUM',
 'pricePTP',
 'priceQUANTILE75',
 'priceQUANTILE25',
 'encode_priceMEAN',
 'encode_priceSTD',
 'encode_priceMIN',
 'encode_priceMAX',
 'encode_priceSUM',
 'encode_pricePTP',
 'encode_priceQUANTILE75',
 'encode_priceQUANTILE25',
 'label',
 'index',
 'price_last_item',
 'encode_brand_last_item',
 'encode_color_last_item',
 'encode_size_last_item',
 'encode_model_last_item',
 'encode_material_last_item',

In [25]:
features = [f for f in df_train_all_exploded_add_last_recall.columns if f not in ['prev_items', 'next_item', 'index', 'label']]
len(features)

104

In [26]:
features

['locale',
 'last_item',
 'recall',
 'last_item_2',
 'idNUNIQUE',
 'idCOUNT',
 'encode_brandNUNIQUE',
 'encode_brandCOUNT',
 'encode_brandMYMODE',
 'encode_colorNUNIQUE',
 'encode_colorCOUNT',
 'encode_colorMYMODE',
 'encode_sizeNUNIQUE',
 'encode_sizeCOUNT',
 'encode_sizeMYMODE',
 'encode_modelNUNIQUE',
 'encode_modelCOUNT',
 'encode_modelMYMODE',
 'encode_materialNUNIQUE',
 'encode_materialCOUNT',
 'encode_materialMYMODE',
 'encode_authorNUNIQUE',
 'encode_authorCOUNT',
 'encode_authorMYMODE',
 'priceMEAN',
 'priceSTD',
 'priceMIN',
 'priceMAX',
 'priceSUM',
 'pricePTP',
 'priceQUANTILE75',
 'priceQUANTILE25',
 'encode_priceMEAN',
 'encode_priceSTD',
 'encode_priceMIN',
 'encode_priceMAX',
 'encode_priceSUM',
 'encode_pricePTP',
 'encode_priceQUANTILE75',
 'encode_priceQUANTILE25',
 'price_last_item',
 'encode_brand_last_item',
 'encode_color_last_item',
 'encode_size_last_item',
 'encode_model_last_item',
 'encode_material_last_item',
 'encode_author_last_item',
 'encode_price_last_

In [28]:
l = df_train_all_exploded_add_last_recall.shape[1]
# shuffled_df_train_all_exploded_add_last_recall.to_csv('./data/shuffled_df_train_all_exploded_add_last_recall_features_{}.csv'.format(l), index=False)
df_train_all_exploded_add_last_recall.to_csv('./data/df_train_all_exploded_add_last_recall_{}.csv'.format(l), index=False)
df_test_all_exploded_add_last_recall.to_csv('./data/df_test_all_exploded_add_last_recall_{}.csv'.format(l), index=False)
print(l)

108


In [2]:
l = 108
shuffled_df_train_all_exploded_add_last_recall = pd.read_csv('./data/shuffled_df_train_all_exploded_add_last_recall_features_{}.csv'.format(l))
df_train_all_exploded_add_last_recall = pd.read_csv('./data/df_train_all_exploded_add_last_recall_{}.csv'.format(l))
df_test_all_exploded_add_last_recall = pd.read_csv('./data/df_test_all_exploded_add_last_recall_{}.csv'.format(l))

In [14]:
df_train_all_exploded_add_last_recall.shape 

(36062490, 108)

In [4]:
features = [f for f in df_train_all_exploded_add_last_recall.columns if f not in ['prev_items', 'next_item', 'index', 'label']]
len(features)

104

lgb

In [3]:
seed = 2023
kfold = 5

In [32]:
# shuffled_index = np.random.permutation(df_train_all_exploded_add_last_recall.index )

In [34]:
# shuffled_df_train_all_exploded_add_last_recall = df_train_all_exploded_add_last_recall.loc[shuffled_index]

In [35]:
# shuffled_df_train_all_exploded_add_last_recall.to_csv('./data/shuffled_df_train_all_exploded_add_last_recall_features_{}.csv'.format(l), index=False)

In [23]:
# features = [f for f in df_train_all_exploded_add_last_recall.columns if f not in ['prev_items', 'next_item', 'index', 'label']]
# len(features)

43

In [5]:
num_features = ['priceMEAN', 'priceSTD', 'priceMIN', 'priceMAX', 'priceSUM', 'pricePTP', 'priceQUANTILE75', 'priceQUANTILE25', 'price_last_item', 'price_last_item_2', 'price_recall']

In [6]:
cat_features = [f for f in features if f not in num_features]

In [7]:
# train = shuffled_df_train_all_exploded_add_last_recall[cat_features]
# test = df_test_all_exploded_add_last_recall[cat_features]
# y = shuffled_df_train_all_exploded_add_last_recall['label'].astype('int64')

# train.shape, test.shape, y.shape

In [19]:
train = shuffled_df_train_all_exploded_add_last_recall[features]
test = df_test_all_exploded_add_last_recall[features]
y = shuffled_df_train_all_exploded_add_last_recall['label'].astype('int64')

train.shape, test.shape, y.shape

((36062490, 104), (3169720, 104), (36062490,))

In [27]:
origin_label = shuffled_df_train_all_exploded_add_last_recall['next_item']
candidate = shuffled_df_train_all_exploded_add_last_recall['recall']
index_list = shuffled_df_train_all_exploded_add_last_recall['index']
origin_label.shape, candidate.shape, index_list.shape

((36062490,), (36062490,), (36062490,))

In [28]:
def get_val_label_index(group):
    label = group['val_origin_label'].iloc[0]
    candidate = group['val_candidate'].tolist()
    try:
        index = candidate.index(label)
        rrank = 1 / (index + 1)
    except:
        rrank = 0
    return pd.Series({'label_rrank': rrank})

In [29]:
def get_train_label_index(group):
    label = group['train_origin_label'].iloc[0]
    candidate = group['train_candidate'].tolist()
    try:
        index = candidate.index(label)
        rrank = 1 / (index + 1)
    except:
        rrank = 0
    return pd.Series({'label_rrank': rrank})

In [30]:
lgb_params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 2 ** 6,
    'max_depth': 8,
    'tree_learner': 'serial',
    'colsample_bytree': 0.8,
    'subsample_freq': 1,
    'subsample': 0.8,
    'num_boost_round': 1000,
    'max_bin': 255,
    'verbose': -1,
    'nthread' : -1,
    'seed': seed,
    'bagging_seed': seed,
    'feature_fraction_seed': seed,
    'early_stopping_rounds': 100,
    # 'device': 'gpu',  # 设置使用 GPU 加速
    # 'gpu_platform_id': 0,  # 设置 GPU 平台 id
    # 'gpu_device_id': 0  # 设置 GPU 设备 id
}

In [31]:
len_candidate_set = 10

In [32]:
fold_size = int(df_train_all_exploded_add_last_recall.shape[0] / len_candidate_set // kfold) * len_candidate_set  # 7212490
index_list_fold = list(range(0, len(df_train_all_exploded_add_last_recall)))
val_indexes = []
start = 0
for i in range(kfold):
    end = min(start + fold_size, len(index_list_fold))
    print(start, end)
    val_indexes.append(index_list_fold[start:end])
    # print(np.min(val_indexes[-1]), np.max(val_indexes[-1]), len(val_indexes[-1]))
    start = end
train_indexes = []
for i, fold in enumerate(val_indexes):
    print(f"Fold {i+1}:")
    print("Validation set:", len(fold))
    train_indexes.append([index for sublist in val_indexes[:i] + val_indexes[i+1:] for index in sublist])
    print("Training set:", len(train_indexes[-1]))


0 7212490
7212490 14424980
14424980 21637470
21637470 28849960
28849960 36062450
Fold 1:
Validation set: 7212490
Training set: 28849960
Fold 2:
Validation set: 7212490
Training set: 28849960
Fold 3:
Validation set: 7212490
Training set: 28849960
Fold 4:
Validation set: 7212490
Training set: 28849960
Fold 5:
Validation set: 7212490
Training set: 28849960


In [33]:
feat_imp_lgb = list()

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros((len(test)))
print(train.shape[1])

# 模型训练
for fold_ in range(kfold):
    trn_idx = train_indexes[fold_]
    val_idx = val_indexes[fold_]
    print("fold n°{}".format(fold_))
    print(len(trn_idx), len(val_idx))
    trn_data = lgb.Dataset(train.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx], label=y.iloc[val_idx])
    
    num_round = 1000
    clf = lgb.train(
        lgb_params,
        trn_data,
        num_round,
        valid_sets=[trn_data, val_data],
        verbose_eval=50,
        early_stopping_rounds=100,
    )

    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx], num_iteration=clf.best_iteration)
    predictions_lgb[:] += clf.predict(test, num_iteration=clf.best_iteration) / kfold
    feat_imp_lgb.append(clf.feature_importance())

    # print('Calculating MRR!')
    # df = pd.DataFrame({
    #         'val_candidate': candidate.iloc[val_idx].values,  # recall
    #         'val_index': index_list.iloc[val_idx].values,  # recall
    #         'val_origin_label': origin_label.iloc[val_idx].values,  # next-item
    #         'val_y_pred': oof_lgb[val_idx]
    #     })
    # df_sorted = df.sort_values(['val_index', 'val_y_pred'], ascending=[True, False]).reset_index(drop=True)
    # df_grouped = df_sorted.groupby('val_index').apply(get_val_label_index)
    # val_mrr = np.mean(df_grouped['label_rrank'].values)
    # print('Fold {}, Val MRR {}.'.format(fold_, val_mrr))

print("AUC score: {}".format(metrics.roc_auc_score(y, oof_lgb)))
print("F1 score: {}".format(metrics.f1_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Precision score: {}".format(metrics.precision_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Recall score: {}".format(metrics.recall_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))

104
fold n°0
28849960 7212490
Training until validation scores don't improve for 100 rounds
[50]	training's auc: 0.840402	valid_1's auc: 0.840657
[100]	training's auc: 0.849153	valid_1's auc: 0.849284
[150]	training's auc: 0.854011	valid_1's auc: 0.853974
[200]	training's auc: 0.856712	valid_1's auc: 0.856491
[250]	training's auc: 0.858759	valid_1's auc: 0.858373
[300]	training's auc: 0.860335	valid_1's auc: 0.859769
[350]	training's auc: 0.86161	valid_1's auc: 0.860855
[400]	training's auc: 0.862713	valid_1's auc: 0.861773
[450]	training's auc: 0.863762	valid_1's auc: 0.862636
[500]	training's auc: 0.8646	valid_1's auc: 0.863281
[550]	training's auc: 0.865293	valid_1's auc: 0.863782
[600]	training's auc: 0.865933	valid_1's auc: 0.864201
[650]	training's auc: 0.866577	valid_1's auc: 0.864655
[700]	training's auc: 0.867215	valid_1's auc: 0.865098
[750]	training's auc: 0.867864	valid_1's auc: 0.865534
[800]	training's auc: 0.868392	valid_1's auc: 0.865857
[850]	training's auc: 0.868948	v

In [ ]:
print("AUC score: {}".format(metrics.roc_auc_score(y, oof_lgb)))
print("F1 score: {}".format(metrics.f1_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Precision score: {}".format(metrics.precision_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))
print("Recall score: {}".format(metrics.recall_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])))

AUC score: 0.8102795953390152
F1 score: 0.002675196277987787
Precision score: 0.8322368421052632
Recall score: 0.0013397514310981196


In [35]:
train_logloss = metrics.log_loss(y, oof_lgb)
train_auc = metrics.roc_auc_score(y, oof_lgb)
train_precision = metrics.precision_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])
train_recall = metrics.recall_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])
train_f1 = metrics.f1_score(y, [1 if i >= 0.5 else 0 for i in oof_lgb])

In [36]:
print("Train Logloss: {:.4f}, AUC: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}".format(train_logloss, train_auc, train_precision, train_recall, train_f1))

Train Logloss: 0.1184, AUC: 0.8670, Precision: 0.7792, Recall: 0.0507, F1 Score: 0.0952


In [37]:
predictions_lgb.shape 

(3169720,)

In [38]:
df = pd.DataFrame({
    'train_candidate': candidate,
    'train_index': index_list,
    'train_origin_label': origin_label,
    'train_y_pred': oof_lgb
})

df_sorted = df.sort_values(['train_index', 'train_y_pred'], ascending=[True, False]).reset_index(drop=True)
df_grouped = df_sorted.groupby('train_index').apply(get_train_label_index)
train_mrr = np.mean(df_grouped['label_rrank'].values)

In [39]:
train_mrr

0.23230824719174525

In [40]:
df = pd.DataFrame({
    'test_candidate': df_test_all_exploded_add_last_recall['recall'].values,
    'test_index': df_test_all_exploded_add_last_recall['index'].values,
    'test_y_pred': predictions_lgb
})

df_sorted = df.sort_values(['test_index', 'test_y_pred'], ascending=[True, False]).reset_index(drop=True)
df_test_preds = df_sorted.groupby('test_index').apply(lambda group : group['test_candidate'].tolist())


In [41]:
df_test_preds

test_index
0         [364726, 375093, 277862, 296535, 448389, 30854...
1         [381461, 381596, 79751, 121801, 440377, 336532...
2         [283983, 243200, 54937, 257230, 330806, 80419,...
3         [170646, 152287, 60178, 230918, 37043, 236653,...
4         [391069, 423381, 399374, 261807, 4017, 303141,...
                                ...                        
316967    [973160, 985530, 1188024, 1178666, 1308289, 97...
316968    [339779, 221172, 255018, 175976, 941877, 27658...
316969    [320241, 111293, 40228, 490183, 262850, 422393...
316970    [134356, 1073088, 180752, 1252507, 387691, 438...
316971    [438197, 491322, 202352, 490567, 66015, 458107...
Length: 316972, dtype: object

In [44]:
recall = 'window'
test_preds_encoded = pickle.load(open('./data/test_preds_{}_one_phase2_encoded.pkl'.format(recall), 'rb'))  # (len_test, 100)

In [45]:
product2id = json.load(open('data/product2id.json', 'r'))
id2product = json.load(open('data/id2product.json', 'r'))
id2product = {int(k): v for k, v in id2product.items()}

In [47]:
df_test = pd.read_csv('data/phase2/sessions_test_task1.csv')

In [46]:
test_res_unencoded = []
for ind, x in tqdm(enumerate(range(len(test_preds_encoded))), total=len(test_preds_encoded)):
    x = df_test_preds.iloc[ind] + test_preds_encoded[ind][len_candidate_set:]
    assert len(x) == 100
    x_unencoded = [id2product[id_] for id_ in x]
    test_res_unencoded.append(x_unencoded)

100%|██████████| 316972/316972 [00:10<00:00, 28995.77it/s]


In [48]:
df_test['next_item_prediction'] = test_res_unencoded
df_test['next_item_prediction'].apply(len).describe()

count    316972.0
mean        100.0
std           0.0
min         100.0
25%         100.0
50%         100.0
75%         100.0
max         100.0
Name: next_item_prediction, dtype: float64

In [50]:
df_test[['locale', 'next_item_prediction']]

,locale,next_item_prediction
0,DE,"[B0B9GJLV2D, B093X59B31, B07SDFLVKD, B0BGC82WV..."
1,DE,"[B0024NKBQE, B084CB7GX9, B08XW4W667, B004P4QFJ..."
2,DE,"[B09Z3FBXMB, B09Z4T2GJ3, B09Z4PZQBF, B09Z4PYG8..."
3,DE,"[B07T6Y2HG7, B07Y1KLF25, B07T5XY2CJ, B07HQ83TF..."
4,DE,"[B0B2JY9THB, B01MRXVY2O, B09B2WLRWX, B08SXLWXH..."
...,...,...
316967,UK,"[B00V6FIFZ0, B07GKP2LCF, B07GKYSHB4, B09QMK82R..."
316968,UK,"[B00M35Y326, B01LX5Y7RG, B00NVMIO02, B085C7TCT..."
316969,UK,"[B08VD6TCP9, B08VDGMBGP, B08VDHH6QF, B08VD5DC5..."
316970,UK,"[B08DTYFYGP, B089CZWB4C, B08W2JJZBM, B08T1ZJYH..."


In [51]:
a = pd.read_parquet('output/2023_32615_MatchModelV2withATTMatchFold0_99.parquet')

In [52]:
a

,locale,next_item_prediction
0,DE,"[B07SDFLVKD, B091CK241X, B093X59B31, B0BGC82WV..."
1,DE,"[B08HQWQ1SK, B084CB7GX9, B08B66B3GB, B08XW4W66..."
2,DE,"[B09Z4PZQBF, B09Z3FBXMB, B09Z4PYG8Q, B09Z4T2GJ..."
3,DE,"[B07HQ83TFF, B07T6Y2HG7, B07Y1KLF25, B07T2NBLX..."
4,DE,"[B0B2JY9THB, B0B2DRKZ6X, B01MRXVY2O, B08YK8FQJ..."
...,...,...
316967,UK,"[B07GKP2LCF, B07GKM97YF, B07GKYSHB4, B00V6FIFZ..."
316968,UK,"[B00NVMIO02, B00M35Y326, B07BMZKCTV, B01LX5Y7R..."
316969,UK,"[B08VD5DC5L, B08VDHH6QF, B08VDGMBGP, B0BK7QC4H..."
316970,UK,"[B0B7M72LFQ, B089CZWB4C, B08W2JJZBM, B08DTYFYG..."


In [56]:
exp_id = '17581'
Fold = 0

In [57]:
model_name = 'LGBFold{}'.format(Fold)

In [58]:
df_test[['locale', 'next_item_prediction']].to_parquet('output/{}_{}_{}.parquet'.format(seed, exp_id, model_name), engine='pyarrow')

In [59]:
predictions_lgb.shape 

(3169720,)

In [60]:
np.save('./output/{}_{}_{}_predictions_lgb.npy'.format(seed, exp_id, model_name), predictions_lgb)

In [61]:
np.save('./output/{}_{}_{}_oof_lgb.npy'.format(seed, exp_id, model_name), oof_lgb)

111